In [3]:
import psycopg2
import os

# Verbindung zur PostgreSQL-Datenbank herstellen
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    user="postgres",
    password=os.environ['DB_PASSWORD'],
    database="Bücher-Datenbank")

# Cursor erstellen
cursor = conn.cursor()

# Log-Tabelle für Bücher erstellen
cursor.execute('''
    CREATE TABLE IF NOT EXISTS log_buchladen (
        id SERIAL PRIMARY KEY,
        produkt TEXT,
        produkt_id INTEGER,
        zeitstempel TIMESTAMP,
        aenderungsart TEXT
    )
''')


# Transaktion bestätigen
conn.commit()

# Verbindung schließen
cursor.close()
conn.close()

In [10]:
# Verbindung zur PostgreSQL-Datenbank herstellen
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    user="postgres",
    password=os.environ['DB_PASSWORD'],
    database="Bücher-Datenbank")

# Cursor erstellen
cursor = conn.cursor()

# Triggerfunktion für die Buch-Tabelle

cursor.execute("""CREATE OR REPLACE FUNCTION buch_logger()
RETURNS TRIGGER
LANGUAGE plpgsql
AS $$
BEGIN
    IF TG_OP = 'INSERT' THEN
        INSERT INTO log_buchladen(produkt,produkt_id, zeitstempel, aenderungsart)
        VALUES ('Buch', new.id, current_timestamp, 'Hinzufügen');
    ELSIF TG_OP = 'UPDATE' THEN
        INSERT INTO log_buchladen(produkt,produkt_id, zeitstempel, aenderungsart)
        VALUES ('Buch', new.id, current_timestamp, 'Ändern');
    ELSIF TG_OP = 'DELETE' THEN
        INSERT INTO log_buchladen(produkt,produkt_id, zeitstempel, aenderungsart)
        VALUES ('Buch', old.id, current_timestamp, 'Löschen');
    END IF;
    RETURN NEW;
END;
$$""", conn)

# Triggerfunktion für die E-Book-Tabelle

cursor.execute("""CREATE OR REPLACE FUNCTION e_book_logger()
RETURNS TRIGGER
LANGUAGE plpgsql
AS $$
BEGIN
    IF TG_OP = 'INSERT' THEN
        INSERT INTO log_buchladen(produkt,produkt_id, zeitstempel, aenderungsart)
        VALUES ('E-Book', new.id, current_timestamp, 'Hinzufügen');
    ELSIF TG_OP = 'UPDATE' THEN
        INSERT INTO log_buchladen(produkt,produkt_id, zeitstempel, aenderungsart)
        VALUES ('E-Book', new.id, current_timestamp, 'Ändern');
    ELSIF TG_OP = 'DELETE' THEN
        INSERT INTO log_buchladen(produkt,produkt_id, zeitstempel, aenderungsart)
        VALUES ('E-Book', old.id, current_timestamp, 'Löschen');
    END IF;
    RETURN NEW;
END;
$$""", conn)

# Triggerfunktion für die Film-Tabelle

cursor.execute("""CREATE OR REPLACE FUNCTION film_logger()
RETURNS TRIGGER
LANGUAGE plpgsql
AS $$
BEGIN
    IF TG_OP = 'INSERT' THEN
        INSERT INTO log_buchladen(produkt,produkt_id, zeitstempel, aenderungsart)
        VALUES ('Film', new.id, current_timestamp, 'Hinzufügen');
    ELSIF TG_OP = 'UPDATE' THEN
        INSERT INTO log_buchladen(produkt,produkt_id, zeitstempel, aenderungsart)
        VALUES ('Film', new.id, current_timestamp, 'Ändern');
    ELSIF TG_OP = 'DELETE' THEN
        INSERT INTO log_buchladen(produkt,produkt_id, zeitstempel, aenderungsart)
        VALUES ('Film', old.id, current_timestamp, 'Löschen');
    END IF;
    RETURN NEW;
END;
$$""", conn)


# Transaktion bestätigen
conn.commit()

# Verbindung schließen
cursor.close()
conn.close()

In [11]:
# Verbindung zur PostgreSQL-Datenbank herstellen
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    user="postgres",
    password=os.environ['DB_PASSWORD'],
    database="Bücher-Datenbank")


# Cursor erstellen

cursor = conn.cursor()


# Trigger für die Buch-Tabelle

cursor.execute("""CREATE TRIGGER buch_log_trigger
               AFTER INSERT OR UPDATE OR DELETE ON buch
               FOR EACH ROW
               EXECUTE FUNCTION buch_logger()""", conn)

# Trigger für die Buch-Tabelle

cursor.execute("""CREATE TRIGGER e_book_log_trigger
               AFTER INSERT OR UPDATE OR DELETE ON ebook
               FOR EACH ROW
               EXECUTE FUNCTION e_book_logger()""", conn)

# Trigger für die Buch-Tabelle

cursor.execute("""CREATE TRIGGER film_log_trigger
               AFTER INSERT OR UPDATE OR DELETE ON film
               FOR EACH ROW
               EXECUTE FUNCTION film_logger()""", conn)


# Transaktion bestätigen
conn.commit()

# Verbindung schließen
cursor.close()
conn.close()

In [12]:
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    user="postgres",
    password=os.environ['DB_PASSWORD'],
    database="Bücher-Datenbank")

# Cursor erstellen
cursor = conn.cursor()

# Triggerfunktion für die Buch-Tabelle

cursor.execute("SELECT * FROM log_buchladen;")
cursor.fetchall()

[(1,
  'Buch',
  0,
  datetime.datetime(2023, 10, 23, 14, 28, 27, 274491),
  'Löschen'),
 (2,
  'Buch',
  None,
  datetime.datetime(2023, 10, 23, 14, 28, 50, 86406),
  'Hinzufügen'),
 (3,
  'Buch',
  0,
  datetime.datetime(2023, 10, 23, 14, 36, 44, 311682),
  'Löschen'),
 (4, 'Buch', 1, datetime.datetime(2023, 10, 23, 14, 36, 58, 92098), 'Löschen'),
 (5,
  'Buch',
  0,
  datetime.datetime(2023, 10, 23, 14, 37, 50, 664746),
  'Hinzufügen'),
 (6,
  'Buch',
  1,
  datetime.datetime(2023, 10, 23, 14, 38, 0, 781494),
  'Hinzufügen'),
 (7, 'Buch', 0, datetime.datetime(2023, 10, 23, 14, 39, 6, 285688), 'Ändern')]